## ChatLab: Chat Experiments, Simplified

💬🔬

In [1]:
import chatlab

chat = chatlab.Chat()

await chat("What kind of bird is a murkrow?")

Murkrow is a Pokémon species in the popular franchise. It is categorized as the "Darkness" Pokémon and it resembles a crow or raven in appearance. However, in real life, there is no bird species called "Murkrow."

In [2]:
from chatlab import Chat, user, system

chat = Chat()

await chat(
    system("You are a data science tutor running inside of a Jupyter "
           "Notebook environment that has the full NumFOCUS/PyData stack."),
    user("Help me work with circulation-grade-6.csv"),
)

Certainly! To get started, let's first load the data from the CSV file "circulation-grade-6.csv". We can use the pandas library for this task. Assuming the file is in the same directory as your Jupyter Notebook, you can use the following code:

``` python
import pandas as pd

data = pd.read_csv('circulation-grade-6.csv')
```

Now that the data is loaded, we can start exploring it and performing various operations. Could you please provide some specific tasks you would like to accomplish or any questions you have about the data?

## Introducing Chat Functions

Where `chatlab` (and OpenAI, really) take it next level is with _Chat Functions_. You can declare a function with a schema then register it in your `Chat` for GPT Models to call. You may recall this kind of behavior from [ChatGPT Plugins](https://noteable.io/chatgpt-plugin-for-notebook/). Now, you can take this even further with your own custom code.

As an example, let's give the large language models the ability to tell time.

In [3]:
from datetime import datetime
from pytz import timezone, all_timezones, utc
from typing import Optional
from pydantic import BaseModel

def what_time(tz: Optional[str] = None):
    '''Current time, defaulting to UTC'''
    if tz is None:
        pass
    elif tz in all_timezones:
        tz = timezone(tz)
    else:
        return 'Invalid timezone'
        
    return datetime.now(tz).strftime('%I:%M %p')

class WhatTime(BaseModel):
    tz: Optional[str]

Let's break this down.

`what_time` is the function we're going to provide access to. Its docstring forms the `description` for the model while the schema comes from the pydantic `BaseModel` called `WhatTime`.

In [4]:
import chatlab

chat = chatlab.Chat()

# Register our function
chat.register(what_time, WhatTime)

{'name': 'what_time',
 'description': 'Current time, defaulting to UTC',
 'parameters': {'title': 'WhatTime',
  'type': 'object',
  'properties': {'tz': {'title': 'Tz', 'type': 'string'}}}}

In [5]:
await chat("What time is it?")

The current time is 06:08 PM.

In [6]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': '06:08 PM', 'name': 'what_time'},
 {'role': 'assistant', 'content': ''}]

In [7]:
await chat("Great. What time is it in Osaka?")

The current time in Osaka is 10:08 AM.

Under the hood, `chatlab` is keeping track of the conversation history for you in `.messages`

In [8]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': '06:08 PM', 'name': 'what_time'},
 {'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Great. What time is it in Osaka?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time',
   'arguments': '{\n  "tz": "Asia/Tokyo"\n}'}},
 {'role': 'function', 'content': '10:08 AM', 'name': 'what_time'},
 {'role': 'assistant', 'content': ''}]

## Outro

Thanks for reading through! Please post issues and reach out. I want to hear your experiences!